# Importar Blibliotecas

In [0]:
import cvxpy as cvx
import copy
from heapq import *
import numpy as np
import itertools
counter = itertools.count() 

# Criação Da Classe

In [0]:
class BBTreeNode():
    def __init__(self, vars = set(), constraints = [], objective=0, bool_vars=set()):
        self.vars = vars
        self.constraints = constraints
        self.objective = objective
        self.bool_vars = bool_vars
        self.children = []
    def buildProblem(self):
        prob = cvx.Problem(cvx.Minimize(self.objective), self.constraints) #criando problema, colocando minimização.
        return prob



    def is_fractional(self):# verificando se é fracionario, e descobrindo os fracionarios.
      self.fracionario = []
      for num in self.bool_vars:
        if not str(round(num.value, 2)).replace('.','').isdigit(): return 
        if(float(round(num.value, 2)) != int(round(num.value, 2))):
          self.fracionario.append(num)
      
      if (len(self.fracionario) == 0):
        return False
      return self.fracionario

    

    def branch(self):
        children = []
        for b in [0,1]:
                n1 = copy.deepcopy(self) #Faz uma copia do nó da instancia
                v = n1.heuristic() #Descobri o numero mais perto do 0.5, usando heuristica.
                n1.constraints.append( v == b ) # adicionando novas restrições binarias
                n1.children = []
                n1.bool_vars.remove(v) #Remove a restrição binaria do bool_vars
                n1.vars.add(v) #adionando as novas restrições a vars
                children.append(n1) # colona o novo nó na lista do filho
        print("seguindo pela variavel com valor: ", round(v.value, 4))  # seguimento das novas restrições           
        return children


    def Is_infinit(self): # verificar se é inviavel, se o valor de z for none
        tester = 0
        z = (list([(v.value) for v in self.bool_vars] + [(v.value) for v in self.vars] ) )
        for i in range(len(z)):
          if(z[i] == None ):
            tester +=1
            continue
        if tester == len(z):
          return True
        else:
          return False


    def heuristic(self): #verifica o mais proximo do 0.5, e retornar esta variavel.
        mais_proximo = 1e30
        for v in self.fracionario:
          if (round(v.value, 4) > 0.5):
            x = range(round((int(v.value) * 100 ), 4) ,50 , -5)
          elif (round(v.value, 4) < 0.5):
            x = range (round((int(v.value) * 100 ), 4), 50, 5)
          elif (round(v.value, 4) == 0.5):
            return v
          if (len(x) <= mais_proximo):
            mais_proximo = v.value
            var_mais_proxima = v
        return var_mais_proxima

          
            
      


    def bbsolve(self): #resolvendo as relaxações, e criando a arvore para resolução.
        print("~~"*32)
        print("Adicionando Nó Pai")
        root = self
        res = root.buildProblem().solve()
        heap = [(res, next(counter), root)]
        bestres = 1e20 # cria um arbitrario muito grande para o dual
        bestnode = root # inicia o pai como o melhor nó
        print(heap)
        nodecount = 0
        print(sorted(list([(v.name(), v.value) for v in root.bool_vars] + [(v.name(), v.value) for v in root.vars] )) )
        while len(heap) > 0:
            print("~~"*32)
            nodecount += 1 
            print("Nós abertos: ", len(heap))
            print("abrindo nó...")
            print("~~"*32)
            print("numero do nó",nodecount)
            _, _, node = heappop(heap)
            prob = node.buildProblem()
            res = prob.solve()
            z = (sorted(list([(v.name(), v.value) for v in node.bool_vars] + [(v.name(), v.value) for v in node.vars] )) )
            for i in range(len(z)):
              if(z[i][1] == None ):
                continue
              print("X{} = {:.4f}".format(i,round(z[i][1],4)))
            tester = node.Is_infinit()
            if (tester):
              print("Infinito, podado por inviabilidade")
              continue
            print("Z = ", round(res,4))
            if prob.status not in ["infeasible", "unbounded"]:
                if res > bestres - 1e-3: #verificando se o Z do nó é maior que meu Dual
                    print("Podado por limitante. Podando o nó.")
                    pass
                elif not node.is_fractional(): #se não for fracionario, logo são todos integral.
                        print("Nova solução integral.")
                        bestres = res
                        bestnode = node
                else: # cria dois novos nó com as novas restrições
                    new_nodes = node.branch()
                    for new_node in new_nodes:
                        heappush(heap, (res, next(counter), new_node ) )  # não comparar os nó que ja foram usados usando o counter
        print("Quantidade de nós descobertos: ", nodecount)      
        return bestres, bestnode

# Manipulando

In [118]:
matriz = []
arquivo = open('problema.txt','r')
for linha in arquivo:
  valores = linha.split()
  matriz.append(valores) #ler do arquivo as linhas e coloca na matriz, para manipulações futuras
arquivo.close()



for i in range(len(matriz)):
  for j in range (len(matriz[i])):
    matriz[i][j] = float(matriz[i][j]) #converte as string lida no arquivo para inteiro
matriz[0][0] = int(matriz[0][0])
matriz[0][1] = int(matriz[0][1])
print(matriz)


[[3, 2], [5.0, 10.0, 8.0], [3.0, 5.0, 2.0, 6.0], [4.0, 4.0, 4.0, 7.0]]


In [119]:

tratamento = matriz.pop(0)
print (tratamento)
#separando variavel e restrição
variaveis = int(tratamento[0])
restricao = int(tratamento[1])
#colocando a lista da função objetivo na variavel

funcao_objetivo = matriz.pop(0)
matriz_restricao = matriz



[3, 2]


In [0]:
x = cvx.Variable(variaveis)
constraints = []
for i in range (restricao):
  constraints += [matriz_restricao[i][:-1]*x >= matriz_restricao[i][len(matriz_restricao[i])-1]] # cria as restrições
constraints +=  [x >= 0, x <=1 ] # relaxação entre 0 e 1, nos reais.
objective = funcao_objetivo * x
bool_vars = {x[i] for i in range(variaveis)} 


In [121]:
root = BBTreeNode(constraints = constraints, objective= objective, bool_vars = bool_vars)
res, sol = root.bbsolve()
print("~~"*32)
print("==========Solução encontrada==========")
print("Z = {:.4f}: ".format(res))
print("variaveis: ")
z = (sorted(list([(v.name(), v.value) for v in sol.bool_vars] + [(v.name(), v.value) for v in sol.vars] )) )
for i in range(len(z)):
  print("X{} = {:.2f}".format(i,round(z[i][1],4)))



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Adicionando Nó Pai
[(11.999999999188262, 0, <__main__.BBTreeNode object at 0x7f6161ba5208>)]
[('var35283[0]', 1.0000000000573852), ('var35283[1]', 0.5000000001357168), ('var35283[2]', 0.24999999969302117)]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Nós abertos:  1
abrindo nó...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
numero do nó 1
X0 = 1.0000
X1 = 0.5000
X2 = 0.2500
Z =  12.0
seguindo pela variavel com valor:  0.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Nós abertos:  2
abrindo nó...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
numero do nó 2
Infinito, podado por inviabilidade
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Nós abertos:  1
abrindo nó...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
numero do nó 3
X0 = 0.7500
X1 = 1.0000
X2 = 0.0000
Z =  13.75
seguindo pela variavel com valor:  0.